# Deep Seek for Math Questions

In [91]:
import pandas as pd
df = pd.DataFrame({'id': [0,0,0,0,0,1], 'a': [None] * 6, 'b':[None] * 6})

In [92]:
vote_counts = melted_df.pivot_table(index=['id', 'vote'], columns='vote_type', values='vote', aggfunc='size', fill_value=0)
vote_counts

,vote_type,b
id,vote,
0,2.0,4
1,1.0,1


In [100]:
# turn table into long format
melted_df = df.melt(id_vars='id', value_vars=['a', 'b'], var_name='vote_type', value_name='vote')
# count votes
vote_counts = melted_df.pivot_table(index=['id', 'vote'], columns='vote_type', values='vote', aggfunc='size', dropna=False, fill_value=0)
final_votes = (vote_counts['a'] * 5 + vote_counts['b']).reset_index()
final_votes.columns = ['id', 'vote', 'weighted_sum']
final_votes

,id,vote,weighted_sum
0,0,NaN,30
1,1,NaN,6


In [101]:
final_votes.loc[final_votes.groupby('id')['weighted_sum'].idxmax()]

,id,vote,weighted_sum
0,0,NaN,30
1,1,NaN,6


In [104]:
final_votes.loc[final_votes.groupby('id')['weighted_sum'].idxmax()][['id', 'vote']]

,id,vote
0,0,NaN
1,1,NaN


In [75]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    negative = False  # to track the presence of a negative sign

    for l in reversed(list(answer)):
        if l == '-' and start and not end:  # check if a negative sign is present before any digits
            negative = True
            end = True
        elif l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True

    if negative:
        out.append('-')  # append negative sign last to maintain order when reversed

    out = reversed(out)
    return ''.join(out)

In [101]:
naive_parse('aafds12-12dafdaf')

'-12'

In [98]:
pd.DataFrame({'a': 1, 'b': None}, index=[0]).groupby('a').agg(lambda x: pd.Series.mode(x).iloc[0] if len(pd.Series.mode(x)) > 0 else None)

,b
a,
1,None


In [47]:
import os
import json
from pathlib import Path
from datetime import datetime
import re, sys, subprocess, gc

import pandas as pd
import numpy as np
import torch
import transformers
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig



model_name = "deepseek-ai/deepseek-math-7b-rl"
config = AutoConfig.from_pretrained(model_name)
config.gradient_checkpointing = True
tokenizer = AutoTokenizer.from_pretrained(model_name)

quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=quantization_config,
                                             device_map="auto")
print(model.dtype)
pipeline = transformers.pipeline("text-generation",
                                 model=model,
                                 tokenizer=tokenizer,
                                 device_map="auto",
                                 config=config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


torch.float16


In [48]:
type(pipeline)

transformers.pipelines.text_generation.TextGenerationPipeline

In [49]:
questions = pd.read_csv('./test.csv')

In [50]:
folder = Path('./MATH/train')
math_train = []
for sub_folder in os.listdir(folder):
    if sub_folder == '.DS_Store':
        continue
    for file in os.listdir(folder / sub_folder):
        if file.endswith('.json'):
            with open(folder / sub_folder / file, 'r') as file:
                data = json.load(file)
                math_train.append(data)
math_train = pd.DataFrame(math_train)
math_train[:3]

,problem,level,type,solution
0,"What is the perimeter, in cm, of quadrilateral...",Level 5,Prealgebra,"Since the given segments are perpendicular, we..."
1,"Four points $B,$ $A,$ $E,$ and $L$ are on a st...",Level 3,Prealgebra,Since the sum of the angles at any point on a ...
2,What is $4\cdot 6+8\cdot 3-28\div 2$?,Level 1,Prealgebra,Recall that multiplication and division should...


In [51]:
math_train['type'].drop_duplicates()

0                   Prealgebra
1205             Number Theory
2074      Intermediate Algebra
3369                   Algebra
5113    Counting & Probability
5884               Precalculus
6630                  Geometry
Name: type, dtype: object

In [52]:
def clean_latex(text):
    """
    convert r"[math]\vec{x} + \vec{y}" to English
    """
    # edge case
    text = re.sub(r'\[math\]', ' LaTex math ', text)
    text = re.sub(r'\[\/math\]', ' LaTex math ', text)
    text = re.sub(r'\\', ' LaTex ', text)

    pattern_to_sub = {
        r'$': '', #Add replace $?
        r'\\mathrm': ' LaTex math mode ',
        r'\\mathbb': ' LaTex math mode ',
        r'\\boxed': ' LaTex equation ',
        r'\\begin': ' LaTex equation ', #should it be parenthesis?
        r'\\end': ' LaTex equation ', #should it be parenthesis?
        r'\\left': ' LaTex equation ', #should it be parenthesis?
        r'\\right': ' LaTex equation ', #should it be parenthesis?
        r'\\(over|under)brace': ' LaTex equation ',
        r'\\text': ' LaTex equation ',
        r'\\vec': ' vector ',
        r'\\var': ' variable ',
        r'\\theta': ' theta ',
        r'\\mu': ' average ',
        r'\\min': ' minimum ',
        r'\\max': ' maximum ',
        r'\\sum': ' + ',
        r'\\times': ' * ',
        r'\\cdot': ' * ',
        r'\\hat': ' ^ ',
        r'\\frac': ' / ',
        r'\\div': ' / ',
        r'\\sin': ' Sine ',
        r'\\cos': ' Cosine ',
        r'\\tan': ' Tangent ',
        r'\\infty': ' infinity ',
        r'\\int': ' integer ',
        r'\\in': ' in ',
    }
    # post process for look up
    pattern_dict = {k.strip('\\'): v for k, v in pattern_to_sub.items()}
    # init re
    patterns = pattern_to_sub.keys()
    pattern_re = re.compile('(%s)' % '|'.join(patterns))

    def _replace(match):
        try:
            word = pattern_dict.get(match.group(0).strip('\\'))
        except KeyError:
            word = match.group(0)
            print('!!Error: Could Not Find Key: {}'.format(word))
        return word
    return pattern_re.sub(_replace, text)

In [53]:
def run_python_code(code):
    try:
        with open('code.py', 'w') as fout:
            fout.write('from sympy import * \n')
            fout.write(code)
        batcmd = 'timeout 7 ' + sys.executable + ' code.py'
        return subprocess.check_output(batcmd, stderr=subprocess.STDOUT, shell=True).decode('utf8')
    except subprocess.CalledProcessError as e:
        return e.output.decode('utf8')  # Return the error output
    except Exception as ex:
        return str(ex)


def process_output(output):
    result = output
    try:
        shell_output = run_python_code(output.split('```')[1][7:])
        shell_number = re.findall(r'(-?\d+)(?!.*\d)', shell_output)[-1]
        code_output = round(float(eval(shell_number))) % 1000
    except:
        code_output = -1
    try:
        result_output = re.findall(r'\\boxed\{(.*)\}', result)
        if not len(result_output):
            result_output = re.findall(r'(-?\d+)(?!.*\d)', result)[-1]
        else:
            result_output = result_output[-1]
        if not len(result_output):
            result_output = -1
        else:
            result_output = round(float(eval(result_output))) % 1000
    except:
        result_output = -1
    return result_output, code_output

In [54]:
questions = math_train[math_train['level'] == 'Level 5'].sample(2)
print('\n\n\n'.join(questions['problem'].values))

The graph of the equation $9x+223y=2007$ is drawn on graph paper with each square representing one unit in each direction. How many of the $1$ by $1$ graph paper squares have interiors lying entirely below the graph and entirely in the first quadrant?



Let $\mathcal{P}$ be the parabola in the plane determined by the equation $y = x^2.$  Suppose a circle $\mathcal{C}$ intersects $\mathcal{P}$ at four distinct points.  If three of these points are $(-28,784),$ $(-2,4),$ and $(13,169),$ find the sum of the distances from the focus of $\mathcal{P}$ to all four of the intersection points.


In [55]:
print('\n-----------------------------------------\n'.join(questions['solution'].values))

There are $223 \cdot 9 = 2007$ squares in total formed by the rectangle with edges on the x and y axes and with vertices on the intercepts of the equation, since the intercepts of the lines are $(223,0),\ (0,9)$.
Count the number of squares that the diagonal of the rectangle passes through. Since the two diagonals of a rectangle are congruent, we can consider instead the diagonal $y = \frac{223}{9}x$. This passes through 8 horizontal lines ($y = 1 \ldots 8$) and 222 vertical lines ($x = 1 \ldots 222$). At every time we cross a line, we enter a new square. Since 9 and 223 are relatively prime, we don’t have to worry about crossing an intersection of a horizontal and vertical line at one time. We must also account for the first square. This means that it passes through $222 + 8 + 1 = 231$ squares.
The number of non-diagonal squares is $2007 - 231 = 1776$. Divide this in 2 to get the number of squares in one of the triangles, with the answer being $\frac{1776}2 = \boxed{888}$.
-----------

In [56]:
question = questions['problem'].iloc[0]
question

'The graph of the equation $9x+223y=2007$ is drawn on graph paper with each square representing one unit in each direction. How many of the $1$ by $1$ graph paper squares have interiors lying entirely below the graph and entirely in the first quadrant?\n'

In [60]:
# tool_instruction = " The answer should be given as a non-negative modulo 1000."

n_repetitions = 3
q_ics = []
raw_results = []
total_results = []
total_answers = []

for q_idx in range(len(questions)):
    print(f"\n\n{q_idx}: {questions['problem'].iloc[q_idx]}")
    results = []
    answers = []
    try:
        combined_messages = None
        for rep_idx in range(n_repetitions):
            print(datetime.now().strftime('%H:%M:%S'))
            if rep_idx > (n_repetitions + 1) / 2:
                problem = clean_latex(questions['problem'].iloc[q_idx])
            else:
                problem = questions['problem'].iloc[q_idx] 
            messages = [{
                "role": "user", 
                "content": 'Problem: ' + problem + "\nGenerate Python code to solve the above problem." +
                '\nUnless complex numbers are mentioned in the problem create SymPy symbols with parameter real=True' +
                    '\n```python\n'
            }]
            query_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
            first_output = pipeline(query_prompt, max_new_tokens=1024, do_sample=True, temperature=0.9, return_full_text=False)
            first_output = '```python\n' + first_output[0]['generated_text']
            # print('Shell input: \n' + first_output.split('```')[1][7:] + '\n')
            shell_output = run_python_code(first_output.split('```')[1][7:])
            print(f'{shell_output=}\n')
            with_output = first_output.split('```output')[0] + f'```output\n{shell_output}\n```'
            message = {
                "role": "assistant",
                "content": f'Answer {rep_idx}:\n```python\n' + with_output
            }
            messages.append(message)
            #torch.cuda.empty_cache()
            #gc.collect()
            if combined_messages is None:
                combined_messages = messages
            else: 
                combined_messages.append(message)
                
        combined_messages.append({
            "role": "user",
            "content": ""
                    
        })
        second_prompt = tokenizer.apply_chat_template(combined_messages, tokenize=False).replace('<｜end▁of▁sentence｜>', '')
        # print(f'{second_prompt=}\n')
        second_output = pipeline(second_prompt, max_new_tokens=1024, do_sample=True, temperature=0.1, return_full_text=False)
        print(f'{second_output=}\n')
        
    except Exception as ex:
        print(ex)
        result_output, code_output = -1, -1 
    raw_results.append(second_output)
    result_output, code_output = process_output(second_output)


    results.append(result_output)
    answers.append(code_output)
    q_ics.append(q_idx)
total_results.append(results)
total_answers.append(answers)

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.




0: The graph of the equation $9x+223y=2007$ is drawn on graph paper with each square representing one unit in each direction. How many of the $1$ by $1$ graph paper squares have interiors lying entirely below the graph and entirely in the first quadrant?

18:19:20


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


shell_output='Traceback (most recent call last):\n  File "/home/daniel/code/math-ai/code.py", line 21, in <module>\n    result = count_squares()\n             ^^^^^^^^^^^^^^^\n  File "/home/daniel/code/math-ai/code.py", line 11, in count_squares\n    y_intercept = solve(x, y)[0].evalf()\n                  ~~~~~~~~~~~^^^\nIndexError: list index out of range\n'

18:19:54


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


...

shell_output='2*sqrt(5) + 13*sqrt(170) + 28*sqrt(785)\n'

18:21:03


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


shell_output='  File "/home/daniel/code/math-ai/code.py", line 2\n    Find the distance of the four points to the focus of the parabola y = x^2.\n         ^^^\nSyntaxError: invalid syntax\n'

18:21:17


--- Logging error ---
Traceback (most recent call last):
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/logging/__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/logging/__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/logging/__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/daniel/mambaforge/envs/pytorch/lib/python3.11/site-packages/ipykernel_launcher.py",

shell_output='957.750000000000\n'

second_output=[{'generated_text': '```python\nfrom sympy import symbols, solve, N\n\ndef sum_of_distances_intersection_points():\n    """Problem: Let $\\mathcal{P}$ be the parabola in the plane determined by the equation $y = x^2.$  Suppose a circle $\\mathcal{C}$ intersects $\\mathcal{P}$ at four distinct points.  If three of these points are $(-28,784),$ $(-2,4),$ and $(13,169),$ find the sum of the distances from the focus of $\\mathcal{P}$ to all four of the intersection points.\n"""\n    x, y = symbols(\'x y\', real=True)\n\n    # The focus of the parabola y = x^2 is at (0, 1/4)\n    focus_x, focus_y = 0, 1/4\n\n    # The given intersection points\n    points = [(-28, 784), (-2, 4), (13, 169)]\n\n    # The sum of the distances from the focus to the intersection points\n    sum_of_distances = 0\n    for point in points:\n        # Calculate the distance using the distance formula\n        sum_of_distances += ((focus_x - point[0])**2 + (focus_y - p

In [13]:
from sympy import symbols, solve, Eq
def solve_triangle():
    # Define the variables
    a, b, c, pa, cp = symbols('a b c pa cp')
    # From the angle bisector theorem, we have: a/b = (c+a)/(2c)
    # Simplify this to get the ratio of a to b\n 
    ratio = solve(Eq(a/b, (c+a)/(2*c)), a/b)[0]
    # Given that M is the midpoint of AD, we have: pa = 2*cp
    # Substitute this into the ratio to get the ratio of cp to pa
    ratio = ratio.subs(a, 2*cp)
    # Simplify the ratio
    ratio = ratio.simplify()
    # The problem asks for the sum of m and n in the fraction m,
    # where m and n are relatively prime positive integers.
    # The ratio we found is in the form m/n. So, m = cp and n = pa.
    m, n = ratio.as_numer_denom()
    # Calculate m+n\n 
    sum_mn = m + n
    return sum_mn
result = solve_triangle()
print(result)

3*c + 2*cp


In [ ]:
# pd.DataFrame({
#     'question': np.repeat(questions.values, n_repetitions),
#     'sympy_answers': total_answers,
#     'llm_answers': total_results,
#     'raw_answers': raw_results
# })

In [54]:
print(len(np.repeat(questions['problem'].values, n_repetitions)))
print(len(raw_results))
print(len(total_results))
print(len(total_answers))

75
25
8
8


In [65]:
questions['problem'].iloc[7]

'When the expression $4(x^2-2x+2)-7(x^3-3x+1)$ is fully simplified, what is the sum of the squares of the coefficients of the terms?'

In [19]:
total_results

[[-1, -1, -1], [-1, 2, -1], [-1, 1, 1]]

In [20]:
total_answers

[[-1, 995, 995], [2, 2, -1], [-1, 2, 2]]

In [22]:
questions['solution'].values

array(['In general, $\\mathbf{M} \\begin{pmatrix} 1 \\\\ 0 \\end{pmatrix}$ is the first column of $\\mathbf{M}$, and $\\mathbf{M} \\begin{pmatrix} 0 \\\\ 1 \\end{pmatrix}$ is the second column of $\\mathbf{M}.$\n\nTaking $\\mathbf{v} = \\begin{pmatrix} 1 \\\\ 0 \\end{pmatrix},$ we get\n\\[-5 \\begin{pmatrix} 1 \\\\ 0 \\end{pmatrix} = \\begin{pmatrix} -5 \\\\ 0 \\end{pmatrix}.\\]Taking $\\mathbf{v} = \\begin{pmatrix} 0 \\\\ 1 \\end{pmatrix},$ we get\n\\[-5 \\begin{pmatrix} 0 \\\\ 1 \\end{pmatrix} = \\begin{pmatrix} 0 \\\\ -5 \\end{pmatrix}.\\]Therefore,\n\\[\\mathbf{M} = \\boxed{\\begin{pmatrix} -5 & 0 \\\\ 0 & -5 \\end{pmatrix}}.\\]',
       'We have that\n\\begin{align*}\nak^3 + bk^2 + ck + d &= 0, \\\\\nbk^3 + ck^2 + dk + a &= 0.\n\\end{align*}Multiplying the first equation by $k,$ we get\n\\[ak^4 + bk^3 + ck^2 + dk = 0.\\]Subtracting the equation $bk^3 + ck^2 + dk + a = 0,$ we get $ak^4 = a.$  Since $a$ is nonzero, $k^4 = 1.$  Then $k^4 - 1 = 0,$ which factors as\n\\[(k - 1)(k + 1)(

In [7]:
propm_prefix = '''Problem:
Find the domain of the expression $\frac{\sqrt{x-2}}{\sqrt{5-x}}$.}

Solution:
The expressions inside each square root must be non-negative. Therefore,
$x-2 \ge 0$, so $x\ge2$, and $5 - x \ge 0$, so $x \le 5$. Also, the denominator
cannot be equal to zero, so $5-x>0$, which gives $x<5$. Therefore, the domain of
the expression is $\boxed{[2,5)}$.
Final Answer: The final answer is $[2,5)$. I hope it is correct.

Problem:
If $\det \mathbf{A} = 2$ and $\det \mathbf{B} = 12,$ then find
$\det (\mathbf{A} \mathbf{B}).$
Solution:
We have that $\det (\mathbf{A} \mathbf{B}) = (\det \mathbf{A})(\det \mathbf{B})
= (2)(12) = \boxed{24}.$
Final Answer: The final answer is $24$. I hope it is correct.
Problem:
Terrell usually lifts two 20-pound weights 12 times. If he uses two 15-pound
weights instead, how many times must Terrell lift them in order to lift the
same total weight?

Solution:
If Terrell lifts two 20-pound weights 12 times, he lifts a total of
$2\cdot 12\cdot20=480$ pounds of weight. If he lifts two 15-pound
weights instead for $n$ times, he will lift a total of $2\cdot15\cdot n=30n$
pounds of weight. Equating this to 480 pounds, we can solve for $n$:
\begin{align*}
30n&=480\\
\Rightarrow\qquad n&=480/30=\boxed{16}
\end{align*}
Final Answer: The final answer is $16$. I hope it is correct.

Problem:
If the system of equations
\begin{align*}
6x-4y&=a,\\
6y-9x &=b.
\end{align*}has a solution $(x, y)$ where $x$ and $y$ are both nonzero,
find $\frac{a}{b},$ assuming $b$ is nonzero.

Solution:
If we multiply the first equation by $-\frac{3}{2}$, we obtain
$$6y-9x=-\frac{3}{2}a.$$Since we also know that $6y-9x=b$, we have
$$-\frac{3}{2}a=b\Rightarrow\frac{a}{b}=\boxed{-\frac{2}{3}}.$$
Final Answer: The final answer is $-\frac{2}{3}$. I hope it is correct.

Problem:
'''

question_easy = 'Beth bakes 4, 2 dozen batches of cokies in a week. If these cookies are shared amongst 16 people equally, how many cookies does each person consume?'
question_hard = 'Each of the three-digits numbers $111$ to $999$ is coloured blue or yellow in such a way that the sum of any two (not necessarily different) yellow numbers is equal to a blue number. What is the maximum possible number of yellow numbers there can be?'
postfix = '\nPlease reason step by step, and put your final answer within \boxed{}.'
postfix_2 =  '\nPlease convert this problem into a set of equations'
tokens = tokenizer.encode(question_hard + postfix_2, return_tensors='pt').to('cuda')
out_tokens = model.generate(tokens, max_length=500)
print(tokenizer.decode(out_tokens[0]))

In [34]:
from sympy import symbols, Rational, floor, sqrt
from sympy import solve, And, Le, Ge
def solve_for_prob():
    x = symbols('x')
    # define the polynomial P(x)
    P = x**2 - 3*x - 9
    # define the condition for the problem
    condition = floor(sqrt(P.subs(x, x))) == sqrt(P.subs(x, floor(x)))
    # define the interval
    interval = And(5 <= x, x <= 15)
    # calculate the number of values of x that satisfy the condition in the interval
    valid_count = 0    
    for i in range(5, 16):
        if condition.subs(x, i): 
            valid_count += 1
        # calculate the total number of values of x in the interval
        total_count = 15 - 5 + 1
        # calculate the probability
        probability = Rational(valid_count, total_count)
        # calculate a, b, c, d, e for the fraction form of the probability
        a = 1
        b = 1
        c = 1
        d = -1
        e = 1
        
solve_for_prob()

AttributeError: 'bool' object has no attribute 'subs'

In [ ]:
˘..